# Calculating Damkohler number for the simulations

## Calculate characteristic reaction time scales assuming first order rates
## Import residence time data


In [1]:
import os

#Import third party libraries
import pandas as pd
import numpy as np

#Import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

#Import data science library
import data_reader.data_processing as proc
import analyses.transient as sta

In [2]:
#Load standard values required to navigate through the datasets
data_dir = "E:/Richards_flow/RF_big_sat_2/"
Regimes = ["Slow", "Medium", "Fast"]
vels = [0.00038,0.0038,0.038]
gw = 1

scdict = proc.masterscenarios("Unsaturated") #master dictionary of all spatially heterogeneous scenarios that were run
ratenames = proc.masterrates("Unsaturated")

#Domains
Trial = list(t for t,values in scdict.items())
#Reactive species of concerns
gvarnames = ["DOC","DO","Ammonium","Nitrate","Nitrogen","TOC"]

### Import residence times/breakthrough times from tracer dataset

In [4]:
chem_path_data = os.path.join(data_dir, "conc_comparison_with_sat_26092021.csv")#03082021.csv")
chemdata = pd.read_csv(chem_path_data)
print(chemdata.columns)
chemdata['Regime'] = chemdata['Regime'].replace({'Equal':'Medium'})
print(chemdata.Regime.unique())

Index(['Trial', 'Variance', 'Anisotropy', 'Regime', 'Chem', 'conc_in',
       'conc_out', 'delconc', 'reldelconc', 'normconc', 'Mean_saturation',
       'Time', 'fraction', 'spatial_normconc_base', 'spatial_reldelconc_base',
       'normconc_spatial_fraction', 'reldelconc_spatial_fraction', 'Da63'],
      dtype='object')
['Medium' 'Fast' 'Slow']


In [5]:
chemdatah = chemdata[chemdata.Trial=="H"]
print(chemdatah[["Trial","Chem","normconc_spatial_fraction"]])

    Trial      Chem  normconc_spatial_fraction
0       H       DOC                        1.0
1       H        DO                        1.0
2       H   Nitrate                        1.0
3       H  Ammonium                        1.0
4       H  Nitrogen                        1.0
5       H       TOC                        1.0
294     H       DOC                        1.0
295     H        DO                        1.0
296     H   Nitrate                        1.0
297     H  Ammonium                        1.0
298     H  Nitrogen                        1.0
299     H       TOC                        1.0
588     H       DOC                        1.0
589     H        DO                        1.0
590     H   Nitrate                        1.0
591     H  Ammonium                        1.0
592     H  Nitrogen                        1.0
593     H       TOC                        1.0


### Calculate pseudo first order reaction rate constant
- Load the chemican concentration profile at steady state
- Identify distance traveled for 63% normalized removal
- Ratio of this normalized removal and corresponding travel time is the reaction rate constant

In [6]:
chemdata["k"] = -1*np.log(chemdata.normconc)/chemdata.Time
print(chemdata[["Regime", "Chem", "k"]].groupby(["Regime", "Chem"]).mean())

                        k
Regime Chem              
Fast   Ammonium  0.063672
       DO        0.869407
       DOC       0.186480
       Nitrate   0.005573
       Nitrogen -0.040161
       TOC       0.085491
Medium Ammonium  0.014225
       DO        0.262340
       DOC       0.048475
       Nitrate   0.027223
       Nitrogen  0.017734
       TOC       0.028070
Slow   Ammonium  0.015641
       DO       -0.028140
       DOC       0.028292
       Nitrate   0.000463
       Nitrogen  0.000927
       TOC       0.008666


In [9]:
thresh = 0.05
steps = [500 * 0.005, 2 * 0.005, 2 * 0.0005]
nc = chemdata[chemdata["normconc"] < thresh].index
for n in nc:
    reg = chemdata.iloc[n]["Regime"]
    t = chemdata.iloc[n]["Trial"]
    g = chemdata.iloc[n]["Chem"]
    inletconc = chemdata.iloc[n]["conc_in"]
    filename = reg+"AR_0_RF-A"+t+"_df.npy"
    concdata = np.load(os.path.join(data_dir, filename))
    conctime, TotalFlow, Headinlettime = sta.conc_time (concdata,0,-1,0,-1, 113, [g],"Unsaturated")        
    tracer_dir = os.path.join("E:/Richards_flow/RF_tr2", reg+"AR_0")
    tracerfile = reg+"AR_0_RF-A"+t+"_df.npy"
    tracerdata = np.load(os.path.join(tracer_dir,tracerfile))
    tracertime, TracerFlow, TracerHeadinlettime = sta.conc_time (tracerdata,0,-1,0,-1, 113, ["Tracer_study"],"Unsaturated")            
    idx5 = np.where(conctime[-1, :, 0]<thresh*inletconc)[0]
    if idx5.size != 0:
        point = idx5[0]
        print(reg,g,point)
        loss = conctime[-1, point, 0]/inletconc
        timidx = np.where(np.round(tracertime[:, point, 0], 3) > 10)
        tim = steps[Regimes.index(reg)] * timidx[0][0]
        k = -1*np.log(loss)/tim
        chemdata.iloc[n, chemdata.columns.get_loc("k")] = k
selected_k = "k"
chemdata["tau"] = -np.log(0.37)/chemdata[selected_k].astype(float)

Medium DO 22
Medium DO 23
Medium DO 25
Medium DO 23
Medium DO 23
Medium DO 23
Medium DO 25
Medium DO 24
Medium DO 24
Medium DO 24
Medium DO 23
Medium DO 24
Medium DO 23
Medium DO 30
Medium DO 35
Medium DO 33
Medium DO 34
Medium DO 41
Medium DO 33
Medium DO 34
Medium DO 39
Medium DO 37
Medium DO 30
Medium DO 40
Medium DO 31
Medium DO 46
Medium DO 78
Medium DO 44
Medium DO 35
Medium DO 69
Medium DO 68
Medium DO 55
Medium DO 54
Medium DO 40
Medium DO 87
Medium DO 46
Medium DO 56
Medium DO 40
Medium DO 82
Medium DO 54
Medium DO 41
Medium DO 54


In [10]:
chemdata["Da"] = chemdata.Time/chemdata.tau
chemdata.to_csv(os.path.join(data_dir, "Da_unsaturated.csv"))